<hr style="border:1px solid gray"> </hr>

# Parameter homotopies

*Documentation*: https://www.juliahomotopycontinuation.org/HomotopyContinuation.jl/stable/solve/

Consider the situation in which one has to solve a specific instance of a *parametrized* family of polynomial systems


$$
P = \{F(x,p)  \mid p \in \mathbb{C}^m\}.
$$

For instance, let $P = \{F(x,y,a,b) \mid (a,b)\in\mathbb{C}^2\}$ where

$$F(x,y,a,b) = \begin{bmatrix} x^2-a \\ xy-a+b \end{bmatrix}.$$



In [ ]:
using HomotopyContinuation
@var x y a b
f = [x^2 - a, x * y - a + b]
F = System(f, variables = [x;y], parameters = [a;b])

For solving systems with parameters one should **first solve the system for a random set of complex parameters $q$**. 

After this one should track the solutions towards the parameters $p$ one is interested in by using the *parameter homotopy*

$$H(x,t) := F(x, (1-t)p + tq)$$

Suppose we are interested in solving the system given by $(a,b)=(3,5)$. Then, following the rule above we solve it as follows.

In [ ]:
random_complex_parameters = randn(ComplexF64,2)

S0 = solve(F; target_parameters = random_complex_parameters)

Then, we track the solutions `S0` towards $(a,b)=(3,5)$ using a parameter homotopy.

In [ ]:
S = solve(F, 
        solutions(S0), 
        start_parameters = random_complex_parameters, 
        target_parameters = [3,5])

<hr style="border:1px solid gray"> </hr>

# Tracking towards many parameters

The above computational scheme is particularly useful when one has to solve a system for many different sets of parameters.

Here is an example: consider the real algebraic variety $V=\{f=0\}\subset\mathbb{R}^2$, where

$$f(x,y) = x^2y + 2xy - y^3,$$

and consider the problem of generating points on $V$. 

A way of getting points on $V$ is to intersect it with varying *real* linear spaces and record the real 
intersection points. 

In the above framework, the coefficients of the linear space would be the parameters.

Let us set up the system in Julia and solve it for a random *complex* linear space.

In [ ]:
using HomotopyContinuation
@var x y
@var a b c # variables for the linear space
p = [a, b, c] # collect the parameters in a vector

F = System([x^2 * y + 2x * y - y^3 + 1; a * x + b * y + c],
            variables = [x;y],
            parameters = [a;b;c]
    )

In [ ]:
# now we solve one particular instance for a,b,c complex. we use this as start system
p₀ = randn(ComplexF64, 3)

complex_result = solve(F, target_parameters = p₀)
S_p₀ = solutions(complex_result)

We track these solutions towards <u>many</u> choices of random real parameters $a,b,c$.

In [ ]:
many = 1000

results = solve(F,
                S_p₀,
                start_parameters = p₀,
                target_parameters = [randn(3) for _ in 1:many]);

We can also pass an option to filter solutions with an additional function. 

Say we only want the real solution points, but not the whole result `struct`.

In [ ]:
points = solve(F,
                S_p₀,
                start_parameters = p₀,
                target_parameters = [randn(3) for _ in 1:many],
                transform_result = (x,p) -> real_solutions(x),
                flatten = true
                );

Here is the result:

In [ ]:
using Plots
points_as_matrix = hcat(points...)
scatter(points_as_matrix[1,:], points_as_matrix[2,:], 
        xlims = (-10,10), ylims = (-10,10), 
        markersize = 2,
        markercolor = :steelblue,
        markerstrokecolor = :steelblue)

<hr style="border:1px solid gray"> </hr>

# Parameter homotopies of composite systems

You can also track a parameter homotopies for composite system. If

$$ f = \begin{bmatrix} ab - q\\  ac - p\end{bmatrix}, \quad g =  \begin{bmatrix}x + y\\ y + 3\\ x + 2\end{bmatrix},$$

and you want to track solutions from random complex parameters to $(p,q) = (2, 3)$, this is how it works:

In [ ]:
@var a b c x y 
@var p q
f = System([a * b - q, a * c - p], variables = [a;b;c], parameters = [p;q])
g = System([x + y, y + 3, x + 2], variables = [x;y])

parameters(f ∘ g)

In [ ]:
complex_parameters = randn(ComplexF64, 2)
S_complex = solve(f ∘ g, 
            target_parameters = complex_parameters, 
            start_system = :total_degree) 

Then, we track the solution in `S_complex` to the parameters $(2,3)$.

In [ ]:
S = solve(f ∘ g, 
        solutions(S_complex), 
        start_parameters = complex_parameters, 
        target_parameters=[2, 3])

It does not matter at which level the parameters are.